## <font color=green> Limpeza e preparação dos dados

In [1]:
import pandas as pd
import numpy as np

- Durante a análise e a modelagaem dos dados, um período significativo de tempo é gasto em sua preparação: carga, limpeza, transformação e reorganização.
- Muitas vezes, o modo como os dados são armazenados em arquivos ou em bancos de dados não constituem o formato correto para uma tarefa em particular.
- Pandas, juntos com os recursos embutidos da linguagem Python, oferecem um conjunto de ferramentas de alto nível, rápido e flexível, para permitir que você manipule os dados, deixando-os no formato correto.

### Tratando dados ausentes

- Dados ausente são comuns em muitas aplicações de análise de dados.
- Para dados numéricos, o pandas utiliza o valor de ponto flutuante NaN (Not a Number) para representá-los.

In [2]:
string_data = pd.Series(['valor 1', 'valor 2', np.nan, 'valor 3'])
string_data

0    valor 1
1    valor 2
2        NaN
3    valor 3
dtype: object

- isnull é o método que devolve valores booleanos informando quais valores estão ausentes

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

- notnull é a negação de isnull

In [4]:
string_data.notnull()

0     True
1     True
2    False
3     True
dtype: bool

- No pandas, referenciamos dados ausentes como NA, que quer dizer Not Available (indisponível).
- É importante fazer a análise nos próprios dados ausentes a fim de identificar problemas em sua coleta ou possíveis distorções provocadas por dados ausentes.

In [5]:
string_data[0] = None
string_data

0       None
1    valor 2
2        NaN
3    valor 3
dtype: object

In [6]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

- dropna é o método que filtra rótulos de eixos, baseado no fato de os valores para cada rótulo terem dados ausentes, com limites variados para a quantidade de dados ausentes a ser tolerado.

In [7]:
string_data.dropna()

1    valor 2
3    valor 3
dtype: object

- drona é equivalente a:

In [8]:
string_data[string_data.notnull()]

1    valor 2
3    valor 3
dtype: object

- fillna é o método que preenche os dados ausentes com algum valor ou utilizando um método de interpolação como 'ffill' ou 'bfill.

In [9]:
string_data.fillna(0)

0          0
1    valor 2
2          0
3    valor 3
dtype: object

In [10]:
string_data.fillna(method='ffill')

0       None
1    valor 2
2    valor 2
3    valor 3
dtype: object

In [11]:
string_data.fillna(method='bfill')

0    valor 2
1    valor 2
2    valor 3
3    valor 3
dtype: object

In [12]:
string_data.notnull()

0    False
1     True
2    False
3     True
dtype: bool

- Com objetos DataFrame, a situação é um pouco mais complexa em relação a exclusão de valores NA por linhas, colunas ou pelos dois.

In [13]:
data = pd.DataFrame([[1, 6.5, 3], [1, np.nan, np.nan], [np.nan, np.nan, np.nan], [np.nan, 6.5, 3]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [14]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


- Passar how='all' descartará apenas as linhas que contenham somente NAs

In [15]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


- Para descartar colunas do mesmo modo, passe axis='columns' ou axis=1

In [16]:
data[4] = np.NaN
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [17]:
data.dropna(how='all', axis=1)

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


- Suponha que queremos manter somente as linhas contendo determinado número de observações. Podemos representar isso com o argumento thresh

In [18]:
data.dropna(thresh=2)

,0,1,2,4
0,1.0,6.5,3.0,NaN
3,NaN,6.5,3.0,NaN


### Removendo duplicatas

In [19]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'], 'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [20]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [21]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


### Substituindo valores

In [22]:
data = pd.Series([1, -999, 2, -999, 3, -1000, 4])
data

0       1
1    -999
2       2
3    -999
4       3
5   -1000
6       4
dtype: int64

- Supondo que os valores de -999 podem ser valores de sentinela para dados ausentes. Para substituí-los por valores NA, podemos usar replace

In [23]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4       3.0
5   -1000.0
6       4.0
dtype: float64

- Para substituir diversos, basta passar os valores numa lista

In [24]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
5    NaN
6    4.0
dtype: float64

- Para usar um substituto diferente para cada valor, passe uma lista deles

In [25]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
5    0.0
6    4.0
dtype: float64

### Detectando e filtrando valores discrepantes

- Filtrar ou transformar valores discrepantes (outliers) é, em boa medida, em uma questão de aplicar operações de array. Considere um DataFrame com alguns dados normalmente distribuídos

In [26]:
data = pd.DataFrame(np.random.randn(1000, 4))
data

,0,1,2,3
0,1.130507,1.488263,-1.282070,-0.204792
1,0.591077,-1.328322,0.420395,0.588469
2,-0.809890,-0.764165,-0.411809,-0.772804
3,-0.670452,-0.587789,1.406915,1.026987
4,0.641791,-0.416728,0.808329,0.908395
...,...,...,...,...
995,1.856219,-1.191524,-1.498318,-1.017301
996,1.025834,1.417592,-2.333211,-0.194665
997,-0.559843,1.539932,-0.570945,-1.753647
998,0.021811,0.704432,-0.534305,-0.871718


In [27]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.050407,0.028581,-0.025342,-0.005179
std,1.006232,1.001140,1.050137,1.021996
min,-3.546045,-2.945722,-2.960806,-3.681865
25%,-0.734890,-0.668024,-0.752436,-0.721785
50%,-0.023592,-0.003778,-0.008145,-0.067217
75%,0.687360,0.726005,0.679199,0.695205
max,2.908475,3.608226,2.788453,3.131224


- Suponha que quiséssemos encontrar os valores que excedessem 3 em valor absoluto em uma das colunas (no nosso caso, coluna 2)

In [28]:
col = data[2]
col[np.abs(col)>3]

Series([], Name: 2, dtype: float64)

- Para selecionar todas as linhas que tenham um valor que exceda 3 ou -3, podemos utilizat o método any um DataFrame

In [29]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
300,-3.079510,-0.650906,0.172465,-1.086944
363,-0.964514,0.518750,1.822537,-3.681865
564,0.689586,-0.014387,0.023096,3.131224
677,-3.546045,-0.823953,0.011040,-0.638146
696,-3.221891,-1.080812,-0.259609,0.666280
797,-3.517268,-0.604442,1.326434,-0.159540
985,0.750984,3.340691,2.100132,0.500332
989,0.789814,3.608226,1.204871,0.832472


- Para eliminar os valores que estejam fora do intervalo de -3 a 3 substituido-os por -3 ou 3 respectivamente

In [30]:
data[np.abs(data)>3] = np.sign(data) * 3

In [31]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.049042,0.027632,-0.025342,-0.004628
std,1.001908,0.998073,1.050137,1.019370
min,-3.000000,-2.945722,-2.960806,-3.000000
25%,-0.734890,-0.668024,-0.752436,-0.721785
50%,-0.023592,-0.003778,-0.008145,-0.067217
75%,0.687360,0.726005,0.679199,0.695205
max,2.908475,3.000000,2.788453,3.000000


### Manipulação de Strings

- Uma string separada por vírgulas pode ser dividida em partes usando split

In [32]:
val = 'a,b, teste'

In [33]:
val.split(',')

['a', 'b', ' teste']

- Com frequência, split é usado em conjunto com strip para remover espaços em branco (incluindo quebra de linha)

In [34]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'teste']

- Replace é usado para substituir as ocorrências de um padrão por outro. É comumente utilizado também para apagar padrões passando uma string vazia

In [35]:
val.replace(',', '::')

'a::b:: teste'

In [36]:
val.replace(',','')

'ab teste'